In [55]:
import csv
import pdb
import numpy as np
import random
random.seed(2000)
import ast
import os
import collections
from importlib import reload
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import cm
from matplotlib.colors import LightSource
import scipy.stats

In [56]:
os.getcwd()

'C:\\Users\\Owner\\Desktop\\dataset\\lumbar-conformal-main\\scripts'

In [57]:
os.chdir('/Users/Owner/Desktop/dataset/lumbar-conformal-main/scripts')
import conformal_demo

In [58]:
num_trials=10
#alphas = np.array([0.01,0.05,0.1,0.15,0.2])
alphas = np.arange(0.01,0.21,0.01)
cal_percent = 0.35

In [59]:
alphas

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 ])

In [60]:
df = pd.read_csv('/Users/Owner/Desktop/mydata.csv')

In [61]:
df.head()

,Unnamed: 0,X1,X2,X3,X4,cal_labels
0,1,0.150923,0.349930,0.384060,0.115087,2
1,4,0.039051,0.180906,0.765951,0.014092,2
2,6,0.639557,0.172963,0.006057,0.181422,1
3,7,0.192730,0.313367,0.155514,0.338389,1
4,8,0.088407,0.104308,0.020772,0.786513,4


In [62]:
def class_conditional_eval(sets, labels, classes=['neg', 'mod', 'mil', 'sev']):
    res = collections.defaultdict()
    for i, v in enumerate(classes):
        class_cond_sets = sets[labels == i]
        n = class_cond_sets.shape[0]
        res[v] = {
            'count': n,
            'coverage': (class_cond_sets[np.arange(n), i].sum() / n) if n > 0 else 0,
            'size': class_cond_sets.sum(1).mean(),
        }
    return res

def size_conditional_eval(sets, labels, sizes=[1, 2, 3, 4]):
    res = collections.defaultdict()
    for i, v in enumerate(sizes):
        size_cond_sets = sets[sets.sum(1) == v]
        n = size_cond_sets.shape[0]
        res[v] = {
            'count': n,
            'coverage': (size_cond_sets[np.arange(n), i].sum() / n) if n > 0 else 0,
        }
    return res

In [63]:
alpha_aps_overall_coverage = collections.defaultdict(list)
alpha_aps_overall_size = collections.defaultdict(list)

alpha_aps_class_cond_count = collections.defaultdict(list)
alpha_aps_class_cond_coverage = collections.defaultdict(list)
alpha_aps_class_cond_size = collections.defaultdict(list)
alpha_aps_size_cond_count = collections.defaultdict(list)
alpha_aps_size_cond_coverage = collections.defaultdict(list)

In [64]:
for alpha in alphas:
    
    aps_overall_coverage = []
    aps_overall_size = []

    aps_class_cond_count = collections.defaultdict(list)
    aps_class_cond_coverage = collections.defaultdict(list)
    aps_class_cond_size = collections.defaultdict(list)
    aps_size_cond_count = collections.defaultdict(list)
    aps_size_cond_coverage = collections.defaultdict(list)
    aps_size_cond_size = collections.defaultdict(list)
    for trial in range(num_trials):
        
        df_cal = df.sample(frac=cal_percent)
        df_val = df.drop(df_cal.index)
        df_cal = df_cal.values
        df_val = df_val.values
        cal_scores = df_cal[:,1:5]
        val_scores = df_val[:,1:5]
        cal_labels = df_cal[:,5]-1
        val_labels = df_val[:,5]-1
        cal_labels = cal_labels.astype('int32')
        val_labels = val_labels.astype('int32')
        
        
            
        # Ordinal APS
        aps_qhat = conformal_demo.get_qhat_ordinal_aps(
            conformal_demo.ordinal_aps_prediction,
            np.copy(cal_scores),
            np.copy(cal_labels),
            alpha
        )
        aps_pred_sets = conformal_demo.ordinal_aps_prediction(np.copy(val_scores), aps_qhat)
        aps_overall_coverage.append(aps_pred_sets[np.arange(val_scores.shape[0]), val_labels].sum() / val_labels.shape[0])
        aps_overall_size.append(aps_pred_sets.sum(1).mean())
        for _class, cond_res in class_conditional_eval(aps_pred_sets, val_labels).items():
            aps_class_cond_count[_class].append(cond_res['count'])
            aps_class_cond_coverage[_class].append(cond_res['coverage'])
            aps_class_cond_size[_class].append(cond_res['size'])
        for _size, cond_res in size_conditional_eval(aps_pred_sets, val_labels).items():
            aps_size_cond_count[_size].append(cond_res['count'])
            aps_size_cond_coverage[_size].append(cond_res['coverage'])

    
    alpha_aps_overall_coverage[alpha] = aps_overall_coverage
    alpha_aps_overall_size[alpha] = aps_overall_size
        
    alpha_aps_class_cond_count[alpha] = aps_class_cond_count
    alpha_aps_class_cond_coverage[alpha] = aps_class_cond_coverage
    alpha_aps_class_cond_size[alpha] = aps_class_cond_size
    alpha_aps_size_cond_count[alpha] = aps_size_cond_count
    alpha_aps_size_cond_coverage[alpha] = aps_size_cond_coverage

In [65]:
aps_agg_coverage = np.array(list(alpha_aps_overall_coverage.values()))
aps_agg_size = np.array(list(alpha_aps_overall_size.values()))

In [66]:
aps_agg_cvg=[]
for x in aps_agg_coverage:
    aps_agg_cvg.append(x.mean())

In [67]:
aps_agg_cvg

[1.0,
 0.9897660818713451,
 0.9853801169590642,
 0.9678362573099415,
 0.955263157894737,
 0.9552631578947368,
 0.9394736842105263,
 0.9251461988304092,
 0.9225146198830408,
 0.9058479532163742,
 0.9032163742690058,
 0.8964912280701753,
 0.8801169590643274,
 0.8807017543859649,
 0.8640350877192983,
 0.8538011695906433,
 0.854970760233918,
 0.8210526315789475,
 0.8160818713450292,
 0.7929824561403508]

In [68]:
aps_agg_set_size=[]
for x in aps_agg_size:
    aps_agg_set_size.append(x.mean())
aps_agg_set_size

[3.9859649122807013,
 3.6131578947368426,
 3.4985380116959064,
 3.3058479532163743,
 3.2219298245614034,
 3.2026315789473685,
 3.1108187134502923,
 3.0514619883040934,
 3.022514619883041,
 2.962573099415205,
 2.9257309941520466,
 2.910818713450292,
 2.817836257309941,
 2.826315789473685,
 2.739181286549708,
 2.6631578947368424,
 2.651169590643275,
 2.4997076023391815,
 2.4713450292397656,
 2.3929824561403508]

In [69]:
aps_sev_coverage = np.array([alpha_aps_class_cond_coverage[alpha]['sev'] for alpha in alphas])
aps_mod_coverage = np.array([alpha_aps_class_cond_coverage[alpha]['mod'] for alpha in alphas])
aps_mil_coverage = np.array([alpha_aps_class_cond_coverage[alpha]['mil'] for alpha in alphas])
aps_neg_coverage = np.array([alpha_aps_class_cond_coverage[alpha]['neg'] for alpha in alphas])

In [70]:
aps_neg_cvg = []
for x in aps_neg_coverage:
    aps_neg_cvg.append(x.mean())
aps_neg_cvg

[1.0,
 0.9977883675464321,
 0.9936382627854938,
 0.9654044686349561,
 0.9508122918568036,
 0.9531242347327493,
 0.9258135332682766,
 0.8996399947094315,
 0.8945678829398229,
 0.875515140210533,
 0.8710779248171108,
 0.8632774916833575,
 0.8498310044704713,
 0.8343308785128022,
 0.8081876192143669,
 0.8006293671824135,
 0.8034072356722479,
 0.7654318475410117,
 0.7690216541633867,
 0.7471316544668876]

In [71]:
aps_mil_cvg = []
for x in aps_mil_coverage:
    aps_mil_cvg.append(x.mean())
aps_mil_cvg

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9989473684210527,
 0.9943992689419938]

In [72]:
aps_mod_cvg = []
for x in aps_mod_coverage:
    aps_mod_cvg.append(x.mean())
aps_mod_cvg

[1.0,
 0.9818240333904349,
 0.968607985944692,
 0.9539413616291551,
 0.9450731614269895,
 0.9441124241413583,
 0.9342393948758249,
 0.9187881242828324,
 0.9126106498381661,
 0.9031929243534098,
 0.9032117727991895,
 0.9017656140763108,
 0.8723237603973999,
 0.8905557833944464,
 0.8696474432067601,
 0.8607197642714108,
 0.867383026318097,
 0.8419375735513531,
 0.8302017951669587,
 0.8122116579785658]

In [73]:
aps_sev_cvg = []
for x in aps_sev_coverage:
    aps_sev_cvg.append(x.mean())
aps_sev_cvg

[1.0,
 0.9767624687025631,
 0.9743363728240189,
 0.9475490831200112,
 0.9201321502773944,
 0.9170628923993481,
 0.8912541473719351,
 0.8731465995382959,
 0.8735522361454408,
 0.8367742284772595,
 0.8297291709040178,
 0.815481700072367,
 0.7840131236637672,
 0.7868124041286821,
 0.7684024538756248,
 0.7319658795271187,
 0.7330397797096886,
 0.6587511673884522,
 0.6542688807961775,
 0.5999509110266862]